In [5]:
import numpy as np
from scipy.stats import norm
from scipy import optimize
import matplotlib.pyplot as plt

CMAP = "cividis"

def getD1(S, K, vol, dt, r):
    return (np.log(S / K) + (r + vol**2 / 2) * dt) / \
        (vol * np.sqrt(dt))

def getCallPrice(spotPrice, strikePrice, vol, dt,riskFreeRate):
    if dt <= 0:
        return max([0, spotPrice - strikePrice])
    if strikePrice <= 0:
        return spotPrice
    d1 = getD1(spotPrice, strikePrice, vol, dt, riskFreeRate)
    d2 = d1 - vol * np.sqrt(dt)
    value = spotPrice * norm.cdf(d1) - strikePrice * \
        np.exp(-riskFreeRate * dt) * norm.cdf(d2)
    delta = norm.cdf(d1)
    return value, delta

def getEquilibriumFee(loanAmount, spotPrice, vol, riskFreeRate, loanTenorInYears):
    def getSwapValue(fee):
        callPrice, _ = getCallPrice(spotPrice, loanAmount, vol, loanTenorInYears, riskFreeRate)
        swapValue = -1*spotPrice + loanAmount + callPrice*(1-fee)
        return swapValue
    
    def minFunc(fee):
        swapValue = getSwapValue(fee)
        return swapValue**2

    initFeeGuess = .5
    feeBnds = (.0, 1.)
    res = optimize.minimize(
        minFunc,
        args=(),
        x0=[initFeeGuess],
        bounds=[feeBnds])
        
    callPrice, callDelta = getCallPrice(spotPrice, loanAmount, vol, loanTenorInYears, riskFreeRate)
    intrinsicVal = max([spotPrice - loanAmount, 0])
    timeVal = callPrice - intrinsicVal
    print(res["fun"])
    if res["success"] and res["fun"][0] < 0.1:
        fee = res['x'][0]
    else:
        fee = None
        
    return fee, callPrice, intrinsicVal, timeVal, callDelta

def getEquilibriumApr(loanAmount, spotPrice, vol, riskFreeRate, loanTenorInYears):
    def getSwapValue(strike):
        callPrice, _ = getCallPrice(spotPrice, strike, vol, loanTenorInYears, riskFreeRate)
        swapValue = -1*spotPrice + loanAmount + callPrice
        return swapValue
    
    def minFunc(strike):
        swapValue = getSwapValue(strike)
        return swapValue**2

    initStrikeGuess = spotPrice
    strikeBnds = (.0000001, None)
    res = optimize.minimize(
        minFunc,
        args=(),
        x0=[initStrikeGuess],
        bounds=[strikeBnds])
        
    strike = res['x'][0]
    callPrice, callDelta = getCallPrice(spotPrice, strike, vol, loanTenorInYears, riskFreeRate)
    intrinsicVal = max([spotPrice - strike, 0])
    timeVal = callPrice - intrinsicVal
    
    if res["success"] and res["fun"][0] < 0.1:
        strike = res['x'][0]
        apr = (strike/loanAmount-1)/loanTenorInYears
    else:
        strike = None
        apr = None
        
    return strike, apr, callPrice, intrinsicVal, timeVal, callDelta

def upfrontFeeHeatmap(params):
    print("ltv; vol; spotPrice; loanAmount; fee; reclaimable; callValue; intrinsicVal; timeVal; callDelta")
    feeRes = []
    for ltv in params["ltvRange"]:
        tmp = []
        params["loanAmount"] = params["spotPrice"]*ltv
        for vol in params["volRange"]:
            args = dict((k, params[k]) for k in ("loanAmount", "spotPrice", "riskFreeRate", "loanTenorInYears"))
            args["vol"] = vol
            (fee, callPrice, intrinsicVal, timeVal, callDelta) = getEquilibriumFee(**args)
            tmp.append(fee)
            print("{:.2f}%; {:.2f}%; {:.2f}; {:.2f}; {:.2f}%; {:.2f}; {:.2f}; {:.2f}; {:.2f}; {:.2f}".format(ltv*100, vol*100, params["spotPrice"], params["loanAmount"], fee*100, 1-fee, callPrice, intrinsicVal, timeVal, callDelta))
        feeRes.append(tmp)
    print(feeRes)
    
    plt.rcParams.update({'font.size': 28})
    fig, ax = plt.subplots(figsize=(15, 15))
    cmap = plt.cm.get_cmap(CMAP)
    ax.imshow(feeRes, cmap=CMAP)
    
    # Show all ticks and label them with the respective list entries
    ax.set_xticks(np.arange(len(params["volRange"])), labels=["{:.0f}%".format(x*100) for x in params["volRange"]])
    ax.set_yticks(np.arange(len(params["ltvRange"])), labels=["{:.0f}%".format(x*100) for x in params["ltvRange"]])

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")
    
    # Loop over data dimensions and create text annotations.
    normColors = plt.cm.colors.Normalize(vmin=np.min(feeRes), vmax=np.max(feeRes))
    for i in range(len(params["ltvRange"])):
        for j in range(len(params["volRange"])):
            bg = cmap(normColors(feeRes[i][j]))
            v = 0 if (bg[0] + bg[1] + bg[2]) / 3 > 0.5 else 1
            c = (v, v, v, 1.)
            ax.text(j, i, "{:.2f}%".format(feeRes[i][j]*100),
                    ha="center", va="center", color=c, fontsize="smaller")
    
    ax.set_title("{} - Upfront Fees (tenor={:.1f}y)".format(params["pair"], params["loanTenorInYears"]))
    ax.set_ylabel("LTV")
    ax.set_xlabel("Vol.")
    fig.tight_layout()
    plt.show()
    fig.savefig('{} aprs.png'.format(params["pair"]), dpi=fig.dpi)
    plt.clf()
    
def aprHeatmap(params):
    print("ltv; vol; spotPrice; loanAmount; apr; strike; callValue; intrinsicVal; timeVal; callDelta")
    aprRes = []
    for ltv in params["ltvRange"]:
        tmp = []
        params["loanAmount"] = params["spotPrice"]*ltv
        for vol in params["volRange"]:
            args = dict((k, params[k]) for k in ("loanAmount", "spotPrice", "riskFreeRate", "loanTenorInYears"))
            args["vol"] = vol
            (strike, apr, callPrice, intrinsicVal, timeVal, callDelta) = getEquilibriumApr(**args)
            tmp.append(apr)
            print("{:.2f}%; {:.2f}%; {:.2f}; {:.2f}; {:.2f}%; {:.2f}; {:.2f}; {:.2f}; {:.2f}; {:.2f}".format(ltv*100, vol*100, params["spotPrice"], params["loanAmount"], apr*100, strike, callPrice, intrinsicVal, timeVal, callDelta))
        aprRes.append(tmp)
    print(aprRes)
    
    plt.rcParams.update({'font.size': 28})
    fig, ax = plt.subplots(figsize=(15, 15))
    cmap = plt.cm.get_cmap(CMAP)
    ax.imshow(aprRes, cmap=CMAP)
    
    # Show all ticks and label them with the respective list entries
    ax.set_xticks(np.arange(len(params["volRange"])), labels=["{:.0f}%".format(x*100) for x in params["volRange"]])
    ax.set_yticks(np.arange(len(params["ltvRange"])), labels=["{:.0f}%".format(x*100) for x in params["ltvRange"]])

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")
    
    # Loop over data dimensions and create text annotations.
    normColors = plt.cm.colors.Normalize(vmin=np.min(aprRes), vmax=np.max(aprRes))
    for i in range(len(params["ltvRange"])):
        for j in range(len(params["volRange"])):
            bg = cmap(normColors(aprRes[i][j]))
            v = 0 if (bg[0] + bg[1] + bg[2]) / 3 > 0.5 else 1
            c = (v, v, v, 1.)
            ax.text(j, i, "{:.2f}%".format(aprRes[i][j]*100),
                    ha="center", va="center", color=c, fontsize="smaller")
    
    ax.set_title("{} - APRs (tenor={:.1f}y)".format(params["pair"], params["loanTenorInYears"]))
    ax.set_ylabel("LTV")
    ax.set_xlabel("Vol.")
    fig.tight_layout()
    plt.show()
    fig.savefig('{} - aprs.png'.format(params["pair"]), dpi=fig.dpi)
    plt.clf()
    

params = {
    "pair": "RPL-USDC",
    "ltvRange": [.1, .25, .5, .75],
    "volRange": [.5, 1., 1.5, 2.],
    "spotPrice": 30.,
    "riskFreeRate": .02,
    "loanTenorInYears": 1./2
}
upfrontFeeHeatmap(params)
aprHeatmap(params)


ltv; vol; spotPrice; loanAmount; fee; reclaimable; callValue; intrinsicVal; timeVal; callDelta
1.82053900667107e-14


<class 'TypeError'>: 'float' object is not subscriptable